# Shadow Model Performance

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as spio
import tensorflow as tf
import tensorflow_addons as tfa
from os.path import join as osj
import pandas as pd
import seaborn as sns
import random
import pickle
import os

## Load results

In [9]:
def load_shadow_results():
    with open(osj("../results_shadow/interpatient_results.pkl"), 'rb') as f:
        results = pickle.load(f)
    with open(osj("../results_shadow/interpatient_loss.pkl"), 'rb') as f:
        loss = pickle.load(f)
    return results, loss

In [10]:
shadow_results, shadow_loss = load_shadow_results()

In [11]:
shadow_results

{'laplace': {0.041: {0.9: {9: {'avg_acc': np.float32(0.9909444),
     'acc': array([0.98641664, 0.9866666 , 0.99974996], dtype=float32),
     'sens': array([1.      , 0.      , 0.994152], dtype=float32),
     'spec': array([0.75780034, 1.        , 1.        ], dtype=float32),
     'prec': array([0.98581374,        nan, 1.        ], dtype=float32),
     'neg_pred_value': array([1.       , 0.9866666, 0.9997389], dtype=float32),
     'false_pos_rate': array([0.24219966, 0.        , 0.        ], dtype=float32),
     'false_neg_rate': array([0.        , 1.        , 0.00584798], dtype=float32),
     'false_det_rate': array([0.01418629,        nan, 0.        ], dtype=float32),
     'f1_score': array([0.9928562,       nan, 0.9970674], dtype=float32)},
    19: {'avg_acc': np.float32(0.9901944),
     'acc': array([0.9852916 , 0.9855417 , 0.99974996], dtype=float32),
     'sens': array([0.99880815, 0.        , 0.994152  ], dtype=float32),
     'spec': array([0.75780034, 0.99885976, 1.        ], d

In [15]:
loss_records = []
result_records = []

#shadow_results, shadow_loss = load_shadow_results()

# # Extract loss
# for epsilon, deltas in shadow_loss.items():
#     for delta, epochs in deltas.items():

#         last_epoch = max(epochs.keys()) if epochs else None  
#         if last_epoch is None:
#             continue 
        
#         batches = epochs[last_epoch]
#         last_batch = max(batches.keys()) if batches else None  
#         if last_batch is None:
#             continue
        
#         loss = batches[last_batch]
#         loss_records.append({
#             "Model": f"Inter-{mechanism}",
#             "Epsilon": epsilon,
#             "Delta": delta,
#             "Loss": loss
#         })

#     dict_mechanism_results = {epsilon: mechanism_results[epsilon] for epsilon in hp_epsilon_values if epsilon in mechanism_results}

# Extract results
for mechanism, epsilons in shadow_results.items():
    for epsilon, deltas in epsilons.items():
        for delta, epochs in deltas.items():
            for epoch, metrics in epochs.items():
                for metric, values in metrics.items():
                    if metric == "avg_acc":
                        continue
                    else:
                        result_records.append({"Model": f"Shadow-{mechanism}", "Epsilon": epsilon, "Delta": delta, "Epoch": epoch, "Metric": metric, "N": values[0], "S": values[1], "V": values[2]})

shadow_results = pd.DataFrame(result_records)
shadow_losses = pd.DataFrame(loss_records)

In [16]:
shadow_results

,Model,Epsilon,Delta,Epoch,Metric,N,S,V
0,Shadow-laplace,0.041,0.9,9,acc,0.986417,0.986667,0.999750
1,Shadow-laplace,0.041,0.9,9,sens,1.000000,0.000000,0.994152
2,Shadow-laplace,0.041,0.9,9,spec,0.757800,1.000000,1.000000
3,Shadow-laplace,0.041,0.9,9,prec,0.985814,NaN,1.000000
4,Shadow-laplace,0.041,0.9,9,neg_pred_value,1.000000,0.986667,0.999739
...,...,...,...,...,...,...,...,...
895,Shadow-bounded_n,0.010,0.3,99,neg_pred_value,0.946970,0.987471,0.999739
896,Shadow-bounded_n,0.010,0.3,99,false_pos_rate,0.234304,0.002363,0.000000
897,Shadow-bounded_n,0.010,0.3,99,false_neg_rate,0.002468,1.000000,0.005964
898,Shadow-bounded_n,0.010,0.3,99,false_det_rate,0.013337,1.000000,0.000000


In [19]:
f1_shadow_results = shadow_results[(shadow_results["Metric"] == "f1_score") & (shadow_results["Epoch"] == 99)]

In [20]:
f1_shadow_results

,Model,Epsilon,Delta,Epoch,Metric,N,S,V
89,Shadow-laplace,0.041,0.9,99,f1_score,0.990644,NaN,0.997067
179,Shadow-laplace,0.021,0.6,99,f1_score,0.992856,NaN,0.997067
269,Shadow-gaussian_a,0.010,0.8,99,f1_score,0.992850,NaN,0.983701
359,Shadow-gaussian_a,0.010,0.4,99,f1_score,0.989201,NaN,0.997067
449,Shadow-gaussian_a,0.041,0.4,99,f1_score,0.991763,NaN,0.994470
539,Shadow-gaussian_a,0.041,0.2,99,f1_score,0.991851,NaN,0.981638
629,Shadow-gaussian_a,0.031,0.9,99,f1_score,0.992856,NaN,0.997067
719,Shadow-gaussian_a,0.021,0.4,99,f1_score,0.992415,NaN,0.997067
809,Shadow-gaussian_a,0.021,0.5,99,f1_score,0.992997,NaN,0.982288
899,Shadow-bounded_n,0.010,0.3,99,f1_score,0.992068,NaN,0.997009


In [12]:
# Debugging: Überprüfe die Struktur von shadow_results
print("shadow_results Struktur:")
for mechanism, epsilons in shadow_results.items():
    print(f"Mechanism: {mechanism}, Epsilons: {epsilons}")
    if epsilons is None:
        raise ValueError(f"Epsilons für Mechanism {mechanism} ist None.")

    for epsilon, deltas in epsilons.items():
        print(f"  Epsilon: {epsilon}, Deltas: {deltas}")
        if deltas is None:
            raise ValueError(f"Deltas für Epsilon {epsilon} ist None.")

        for delta, epochs in deltas.items():
            print(f"    Delta: {delta}, Epochs: {epochs}")
            if epochs is None:
                raise ValueError(f"Epochs für Delta {delta} ist None.")

            for epoch, metrics in epochs.items():
                print(f"      Epoch: {epoch}, Metrics: {metrics}")
                if metrics is None:
                    raise ValueError(f"Metrics für Epoch {epoch} ist None.")
                

shadow_results Struktur:
Mechanism: laplace, Epsilons: {0.041: {0.9: {9: {'avg_acc': np.float32(0.9909444), 'acc': array([0.98641664, 0.9866666 , 0.99974996], dtype=float32), 'sens': array([1.      , 0.      , 0.994152], dtype=float32), 'spec': array([0.75780034, 1.        , 1.        ], dtype=float32), 'prec': array([0.98581374,        nan, 1.        ], dtype=float32), 'neg_pred_value': array([1.       , 0.9866666, 0.9997389], dtype=float32), 'false_pos_rate': array([0.24219966, 0.        , 0.        ], dtype=float32), 'false_neg_rate': array([0.        , 1.        , 0.00584798], dtype=float32), 'false_det_rate': array([0.01418629,        nan, 0.        ], dtype=float32), 'f1_score': array([0.9928562,       nan, 0.9970674], dtype=float32)}, 19: {'avg_acc': np.float32(0.9901944), 'acc': array([0.9852916 , 0.9855417 , 0.99974996], dtype=float32), 'sens': array([0.99880815, 0.        , 0.994152  ], dtype=float32), 'spec': array([0.75780034, 0.99885976, 1.        ], dtype=float32), 'prec'